In [1]:
import requests
import pandas as pd
import pymysql
import sqlalchemy as db
from sqlalchemy import create_engine

### 1. Importing relevant data from database and cleaning it

In [13]:
# creating the connection with database

sqlitedb_path = '../data/raw/raw_data_project_m1.db'

conn_str = f'sqlite:///{sqlitedb_path}'

engine = create_engine(conn_str, pool_pre_ping=True)
connection = engine.connect()
metadata = db.MetaData()

In [14]:
# creating dataframe from sql query to keep the data we'll need

data = pd.read_sql_query("""
SELECT career_info.uuid,
career_info.normalized_job_code,
country_info.country_code,
personal_info.age_group,
personal_info.age
FROM career_info
JOIN country_info
ON country_info.uuid = career_info.uuid
JOIN personal_info
ON personal_info.uuid = career_info.uuid;
""", engine)

In [12]:
data

,uuid,normalized_job_code,country_code,age_group,age
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,None,AT,40_65,61 years old
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,AT,40_65,57 years old
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,AT,26_39,32 years old
3,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,AT,40_65,45 years old
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,AT,40_65,41 years old
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,847165cfda6b1dc82ae22b967da8af2f,SK,26_39,37 years old
9645,39f989f0-db52-0133-8482-0a81e8b09a82,a4d5b8b38f9513825d0d94a981ebe962,SK,40_65,53 years old
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,None,SK,juvenile,1992
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,775190277a849cba701b306a7b374c0a,SK,40_65,47 years old


In [13]:
data['age_group'].unique()

array(['40_65', '26_39', 'juvenile', '14_25'], dtype=object)

In [14]:
age_group = data['age_group'].tolist()

In [15]:
clean_age_group = [i.replace('_', '-') for i in age_group]
properly_age_group = [i.replace('juvenile', '14-25') for i in clean_age_group]

In [16]:
data['age_group'] = properly_age_group

In [17]:
# cleaning age column to keep it simple, readable

age_column = [i.strip(' years old') for i in data['age']]

data['age'] = age_column

In [18]:
# some of the values are the birth year so i'm going to calculate the age these people had in 2016 according to the rest of the ages in age column

def calculating_age(year):
    if len(year) == 4:
        return 2016 - round(int(year))
    else:
        return year

data['age'] = data['age'].apply(calculating_age)

In [19]:
# now that we have all values as ages i'm calculating the age these people have now (only 4 years more because 2021 just started)

data['age'] = data['age'].apply(lambda x : int(x) + 4)

In [20]:
data

,uuid,normalized_job_code,country_code,age_group,age
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,None,AT,40-65,65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,AT,40-65,61
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,AT,26-39,36
3,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,AT,40-65,49
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,AT,40-65,45
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,847165cfda6b1dc82ae22b967da8af2f,SK,26-39,41
9645,39f989f0-db52-0133-8482-0a81e8b09a82,a4d5b8b38f9513825d0d94a981ebe962,SK,40-65,57
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,None,SK,14-25,28
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,775190277a849cba701b306a7b374c0a,SK,40-65,51


In [21]:
data.sort_values(by='age')

,uuid,normalized_job_code,country_code,age_group,age
7219,4be4aeb0-d8cf-0133-69ce-0a81e8b09a82,None,IT,14-25,18
2089,40ff4c50-da49-0133-3fd1-0a81e8b09a82,None,DK,14-25,18
4849,6a108e40-d90b-0133-a71c-0a81e8b09a82,None,GB,14-25,18
7521,c21128b0-dcac-0133-eb17-0a81e8b09a82,None,IT,14-25,18
6843,b169f5a0-d982-0133-7792-0a81e8b09a82,None,IT,14-25,18
...,...,...,...,...,...
2217,a5f22940-da62-0133-3a08-0a81e8b09a82,996da29c84f5eed00da40c230f4c4078,ES,40-65,69
4568,89ef86c0-dabe-0133-c088-0a81e8b09a82,None,FR,40-65,69
5500,fef72890-dbf6-0133-9abe-0a81e8b09a82,None,GB,40-65,69
373,e3efe070-da41-0133-5bfc-0a81e8b09a82,None,BG,40-65,69


In [22]:
#creating new age groups to these new ages

age_group = data['age_group'].tolist()

In [23]:
new_age_group = []

for i in age_group:
    if i == '14-25':
        new_age_group.append(i.replace('14-25', '18-29'))
    elif i == '26-39':
        new_age_group.append(i.replace('26-39', '30-49'))
    else:
        new_age_group.append(i.replace('40-65', '50-69'))

In [24]:
data['age_group'] = new_age_group

In [25]:
data

,uuid,normalized_job_code,country_code,age_group,age
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,None,AT,50-69,65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,861a9b9151e11362eb3c77ca914172d0,AT,50-69,61
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,AT,30-49,36
3,15626d40-db13-0133-ea5c-0a81e8b09a82,049a3f3a2b5f85cb2971ba77ad66e10c,AT,50-69,49
4,24954a70-db98-0133-4a64-0a81e8b09a82,f4b2fb1aa40f661488e2782b6d57ad2f,AT,50-69,45
...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,847165cfda6b1dc82ae22b967da8af2f,SK,30-49,41
9645,39f989f0-db52-0133-8482-0a81e8b09a82,a4d5b8b38f9513825d0d94a981ebe962,SK,50-69,57
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,None,SK,18-29,28
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,775190277a849cba701b306a7b374c0a,SK,50-69,51


### 2. Connecting API and extracting info

In [26]:
# now we're going to create a list with unique job codes

job_code_list = data['normalized_job_code'].unique().tolist()

In [27]:
# from these codes, we're creating urls to create the connection with API 

url_list = []

for i in job_code_list:
    url_list.append(f'http://api.dataatwork.org/v1/jobs/{i}')

In [28]:
# here we'll have a list with the json data we're getting from the connection with the API

lst = []

for url in url_list:
    response = requests.get(url)
    json_data = response.json()
    lst.append(json_data)

In [29]:
job_titles_df = pd.DataFrame(lst)
job_titles_df

,error,uuid,title,normalized_job_title,parent_uuid
0,"{'code': 404, 'message': 'Cannot find job with...",NaN,NaN,NaN,NaN
1,NaN,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056
2,NaN,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator,data coordinator,0b9dd32a367f4562ec77b993053d1910
3,NaN,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer,database developer,b90ca4df5690002377a7b0f1f3d40781
4,NaN,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist,data entry specialist,bf88358c56bb6cbe7eabec38ae333d19
...,...,...,...,...,...
152,NaN,b0fa6ede410f50b82ab74f5a705fe699,Analytical Data Miner,analytical data miner,9656fa2dc39e9643c00a45858e0117d0
153,NaN,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant,data analysis assistant,325e2251e20170928426473156bd3c48
154,NaN,c1fb1a01b78373ac2153c66fa08d16dc,Data Examination Clerk,data examination clerk,da412504dd7b130414b7bbfa2acd563b
155,NaN,05bb9a333a66d6eb151e253623efe1c0,Data Entry Clerk,data entry clerk,bf88358c56bb6cbe7eabec38ae333d19


In [30]:
# now we are merging our main df with this one to have the job titles instead of job codes

merged_db = data.merge(job_titles_df, left_on='normalized_job_code', right_on='uuid')

In [31]:
# dropping columns we won't need

cleaning_df = merged_db.drop(columns = ['normalized_job_code', 'error', 'uuid_y', 'normalized_job_title', 'parent_uuid', 'age'])

In [32]:
# changing null values to string Unemployed

first_almost_proper_df = cleaning_df.fillna('Unemployed')

In [33]:
first_almost_proper_df

,uuid_x,country_code,age_group,title
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,50-69,Unemployed
1,83127080-da3d-0133-c74f-0a81e8b09a82,AT,30-49,Unemployed
2,b50dbb80-da53-0133-8956-0a81e8b09a82,AT,30-49,Unemployed
3,9949c4c0-da5f-0133-c832-0a81e8b09a82,AT,18-29,Unemployed
4,69f1f400-dc5f-0133-ad9b-0a81e8b09a82,AT,50-69,Unemployed
...,...,...,...,...
9644,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,IT,50-69,Data Capture Clerk
9645,c8c33390-da69-0133-063a-0a81e8b09a82,PL,50-69,Data Capture Clerk
9646,d27d24d0-d9b1-0133-03d4-0a81e8b09a82,PL,18-29,Data Capture Clerk
9647,529f3080-d99a-0133-1b7b-0a81e8b09a82,PT,50-69,Data Capture Clerk


### 3. Web scraping and finishing processed dataframe

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content

In [15]:
soup = BeautifulSoup(html, 'html.parser')

In [16]:
table = soup.find('table')

In [17]:
items = [x.text for x in table.find_all('td')]
clean_items = [i.strip('\n') for i in items]

In [18]:
countrys = []
country_codes = []

for i in clean_items:
    if i.startswith('('):
        country_codes.append(i[1:-1])
    else:
        countrys.append(i)

In [19]:
country_dict = dict(zip(countrys, country_codes))

In [20]:
country_dict['Great Britain'] = 'GB' #wrangling

In [21]:
country_df = pd.DataFrame.from_dict(country_dict, orient='index')

In [22]:
country_df = country_df.reset_index() #wrangling from now

In [23]:
country_df[0][1] = 'GR'

In [24]:
country_df = country_df.rename(columns={'index':'country', 0:'country_codes'})

In [25]:
merging_countries = country_df.merge(first_almost_proper_df, left_on='country_codes', right_on='country_code')

NameError: name 'first_almost_proper_df' is not defined

In [26]:
cleaning_df = merging_countries.drop(columns = ['country_codes', 'country_code', 'uuid_x'])

NameError: name 'merging_countries' is not defined

In [48]:
cleaning_df

,country,age_group,title
0,Belgium,50-69,Unemployed
1,Belgium,50-69,Unemployed
2,Belgium,18-29,Unemployed
3,Belgium,50-69,Unemployed
4,Belgium,30-49,Unemployed
...,...,...,...
9644,Great Britain,50-69,Data Analysis Assistant
9645,Great Britain,50-69,Data Analysis Assistant
9646,Great Britain,30-49,Data Examination Clerk
9647,Great Britain,30-49,Data Examination Clerk


In [49]:
processed_df = cleaning_df.rename(columns = {'country' : 'Country', 'age_group' : 'Age Group', 'title' : 'Job Title'})

In [50]:
processed_df.to_csv('data/processed/processed_df.csv')

### 4. Data aggregation to add Quantity and Percentage columns

In [51]:
pd.DataFrame(processed_df.groupby(['Country', 'Job Title'])['Age Group'].value_counts())

Age Group
Country Job Title                                          Age Group           
Austria Automatic Data Processing Customer Liaison (ADP... 30-49              1
                                                           50-69              1
        Automatic Data Processing Planner                  50-69              2
        Automatic Data Processing Systems Security Spec... 30-49              1
                                                           50-69              1
...                                                                         ...
Sweden  SQL Database Administrator                         50-69              1
        Survey Data Technician                             50-69              1
        Unemployed                                         18-29             35
                                                           50-69             26
                                                           30-49             20

[2954 rows x 1 columns]

In [52]:
adding_qt_col = processed_df.value_counts(['Country', 'Job Title', 'Age Group']).reset_index(name='Quantity')

In [53]:
adding_qt_col

,Country,Job Title,Age Group,Quantity
0,Great Britain,Unemployed,50-69,273
1,France,Unemployed,50-69,235
2,Germany,Unemployed,50-69,230
3,Germany,Unemployed,18-29,219
4,Spain,Unemployed,50-69,197
...,...,...,...,...
2949,Italy,Director of Data Operations,18-29,1
2950,Italy,Director of Data Operations,30-49,1
2951,Italy,Electronic Data Interchange System Developer (...,18-29,1
2952,Italy,Electronic Data Interchange System Developer (...,50-69,1


In [54]:
def percentage(dataframe):
    adding_qt_col['Percentage'] = (adding_qt_col['Quantity'] / 
                      adding_qt_col['Quantity'].sum()) * 100
    return adding_qt_col

In [55]:
complete_df = adding_qt_col

In [56]:
complete_df.to_csv('data/results/main_df.csv')

In [57]:
complete_df

,Country,Job Title,Age Group,Quantity
0,Great Britain,Unemployed,50-69,273
1,France,Unemployed,50-69,235
2,Germany,Unemployed,50-69,230
3,Germany,Unemployed,18-29,219
4,Spain,Unemployed,50-69,197
...,...,...,...,...
2949,Italy,Director of Data Operations,18-29,1
2950,Italy,Director of Data Operations,30-49,1
2951,Italy,Electronic Data Interchange System Developer (...,18-29,1
2952,Italy,Electronic Data Interchange System Developer (...,50-69,1


In [12]:
unemployement_qt = 0
employement_qt = 0

for i in complete_df['Job Title']:
    while i == 'Unemployed':
        unemployement_qt += 1
    else:
        employement_qt += 1
        
    
    


NameError: name 'complete_df' is not defined